In [1]:
import torch
import torch.nn.functional as F 
import matplotlib.pyplot as plt # for making figures 
%matplotlib inline 

In [2]:
# read in all the words 
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [3]:
len(words)

32033

In [4]:
# build the vocabulary of characters and mappings to/from integers 
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0 
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [5]:
# build the dataset 

block_size = 3 # context length: how many characters do we take to predict the next one? 
X, Y = [], [] 
for w in words:
    
    # print(w)
    context = [0] * block_size 
    for ch in w + '.':
        ix = stoi[ch] 
        X.append(context)
        Y.append(ix)
        # print(''.join(itos[i] for i in context), '---->', itos[ix])
        context = context[1:] + [ix] # crop and append 
        
X = torch.tensor(X)
Y = torch.tensor(Y)

In [6]:
X.shape, X.dtype, Y.shape, Y.dtype 

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [7]:
# build the dataset 

block_size = 3 # context length: how many characters do we take to predict the next one ? 

def build_dataset(words):
    X, Y = [], [] 
    for w in words: 
        
        # print(w)
        context = [0] * block_size 
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '--->', itos[ix])
            context = context[1:] + [ix] # crop and append 
            
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y 

import random 
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [8]:
C = torch.randn((27, 2))

In [9]:
emb = C[X]
emb.shape 

torch.Size([228146, 3, 2])

In [10]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [11]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)

In [12]:
h 

tensor([[-0.8006,  0.3255,  0.9803,  ..., -0.0978, -0.9948,  0.5538],
        [-0.8585, -0.5499,  0.9903,  ...,  0.4158,  0.9678, -0.9878],
        [-0.9277, -0.9988,  0.5238,  ...,  0.3748,  0.2136,  0.9991],
        ...,
        [ 0.4504,  0.9728,  0.9270,  ..., -0.5798,  0.7510, -0.9538],
        [ 0.4908,  0.9303,  0.2076,  ..., -0.5083,  0.4391, -0.9811],
        [-0.6646, -0.2395,  0.9955,  ...,  0.5864,  1.0000, -0.9996]])

In [13]:
h.shape

torch.Size([228146, 100])

In [14]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [15]:
logits = h @ W2 + b2 

In [16]:
logits.shape 

torch.Size([228146, 27])

In [17]:
counts = logits.exp()

In [18]:
prob = counts / counts.sum(1, keepdim=True)

In [19]:
prob.shape

torch.Size([228146, 27])

In [23]:
Y.shape

torch.Size([228146])

In [24]:
loss = -prob[torch.arange(32), Y[0]].log().mean()
loss 

tensor(17.2333)

In [25]:
# ------------ now made respectable :) --------------

In [26]:
Xtr.shape, Ytr.shape # dataset 

(torch.Size([182625, 3]), torch.Size([182625]))

In [27]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility 
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [28]:
sum(p.nelement() for p in parameters) # number of parameters in total 

11897

In [29]:
for p in parameters:
    p.requires_grad = True 